<a id="setup"></a>
## LangChain memory with watsonx.ai models

This notebook contains sample code for using *ConversationBufferMemory* with models included in watsonx.ai


### Install dependecies

In [ ]:
!pip install pip install ibm-watson-machine-learning --upgrade
!pip install langchain | tail -n 1
!pip install langchain-community
!pip install -U langchain-ibm

### Define the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [ ]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

url = credentials['url']
apikey = credentials['apikey']

### Define the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [ ]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

### Import required packages

In [ ]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
#from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from langchain_ibm import WatsonxLLM


from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from langchain.llms.utils import enforce_stop_tokens

In [ ]:
# If you want to change model types, you can look them up with this line
print([model.name for model in ModelTypes])

### Create the LangChain model object

In [ ]:
# We will use the llama model

# Experiment with different models. We noticed that Flan produces a more concise ouput, but llama - more descriptive and better quality#model_id_1 = "meta-llama/llama-2-70b-chat"
#model_id_1 = "google/flan-ul2"
model_id_1 = "meta-llama/llama-2-13b-chat"

model1_parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 300,
    "min_new_tokens": 1,
    "top_k": 50,
    "top_p": 1
}

# WML model objects - will be used to create models for the LangChain API
# Create the model objects that will be used by LangChain

current_llm = WatsonxLLM(
    model_id=model_id_1,
    url=url,
    project_id=project_id,
    params=model1_parameters,
    apikey=apikey)

In [ ]:
# Create the memory object
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=current_llm, 
    memory = memory,
    verbose=True
)

In [ ]:
# Uncomment and run these lines if you want to clear the memory without stopping the notebook (use if you provide different user input in the cell below)
memory.clear()
#print(memory.buffer)

In [ ]:
# Example of a first quesiton from the user
user_input = "From the following customer complaint, extract 3 factors that caused the customer to be unhappy. \
                            Put each factor on a new line. Customer complaint: I am writing you this statement to delete the \
                            following information on my credit report. The items I need deleted are listed in the report. \
                            I am a victim of identity thief, I demand that you remove these errors to correct my report immediately! \
                            I have reported this to the federal trade commission and have attached the federal trade commission affidavit. \
                            Now that I have given you the following information, please correct my credit report or I shall proceed with involving my attorney! \
                            Numbered list of complaints:"

#user_input="Right now I am bothered! I have attempted to be patient however it is hard to be patient when you feel that you are continually being overlooked by somebody. I think you fail to remember that \Consumer detailing organizations have expected an essential part in amassing and assessing customer credit and other data on shoppers. The XXXX XXXX  is reliant on the reasonable and precision."

# Invoke the LLM
conversation.predict(input=user_input)


In [ ]:
# Example of a second quesiton from the user
user_input = "Does the numbered list of complaints contain a statement about identity fraud? Provide a short answer: yes or no."
conversation.predict(input=user_input)

In [ ]:
# For debugging, print the history of the conversation
print(memory.buffer)

### Authors: 
 **Elena Lowery**, Data and AI Architect
### Updated by:
**Andre de Waal**, Learning Content Developer